In [1]:
import numpy as np 
import pandas as pd
# import os
# import matplotlib.pyplot as plt
# from PIL import Image, ImageDraw, ImageEnhance
# from tqdm.notebook import tqdm
from tqdm import tqdm

# import cv2
# import re
# import time

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train
# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.parallel import DataParallelModel, DataParallelCriterion

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim


In [2]:
import torchvision.models as models
resnet101 = models.resnet101(progress=False, pretrained=True)

In [3]:
# device = torch.device('cpu')
device = torch.device('cuda')
retinanet = model.resnet101(num_classes=2, device=device)
retinanet.load_state_dict(resnet101.state_dict(), strict=False)
retinanet.to(device)

num_features_in of ResidualAfterFPN : 256


ResNet(
  (bn0): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [4]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 24
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
    pin_memory=True,
    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.28s)
creating index...
index created!


In [5]:
criterion = PapsLoss(device)
criterion = criterion.to(device)
retinanet.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(retinanet.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [6]:
saved_dir = '../trained_models/resnet101_320/'
s_epoch= 0
e_epoch= 160

In [7]:
if os.path.isfile(saved_dir + 'model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'model.pt')
    retinanet.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

In [8]:
retinanet.regressionModel.output.bias

Parameter containing:
tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True)

In [9]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=retinanet, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

0th epochs loss 1.0017712081632306 lr 1e-08 


In [10]:
# state = {
#     'epoch': epoch,
#     'state_dict': retinanet.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : total_loss/(step+1)
# }
# torch.save(state, saved_dir + 'model.pt')

In [11]:
# state = {
#     'epoch': 0,
#     'state_dict': retinanet.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [12]:
# state = torch.load(saved_dir + 'all_model.pt')
# retinanet.load_state_dict(state['state_dict'])
# optimizer.load_state_dict(state['optimizer'])

In [13]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.11s)
creating index...
index created!


In [14]:
saved_dir

'../trained_models/resnet101_320/'

In [15]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=retinanet, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.5)  

100%|██████████| 3739/3739 [02:21<00:00, 26.36it/s]

No object detected
